In [4]:
legacy_query = """SELECT DISTINCT
PRODUCT_DETAILS.[NAME],
PRODUCT_DETAILS.[DESCRIPTION],
PRODUCT_DETAILS.[ITEM_GTIN],
SUM(SHIPMENT_DATA.[VOL_STAT_UNIT])/1000 AS TOTAL_DELIVERED_MSU1,
SUM(SHIPMENT_DATA.[VOL_BASE_UNIT])/1000 AS TOTAL_DELIVERED_MSU2,
SUM(SHIPMENT_DATA.[VOL_PLAN_UNIT])/1000 AS TOTAL_DELIVERED_MSU3,
SUM(SHIPMENT_DATA.[VOL_TRNXL_UNIT])/1000 AS TOTAL_DELIVERED_MSU4,
    GEO_LIST.COUNTRY
FROM
WMW.DBO.PRODUCT_D AS PRODUCT_DETAILS
INNER JOIN WMW.DBO.SHIP_PROD_F AS SHIPMENT_DATA ON PRODUCT_DETAILS.PRODUCT_SID = SHIPMENT_DATA.PRODUCT_SID
INNER JOIN WMW.DBO.GEO_D AS GEO_LIST ON GEO_LIST.GEO_SID = SHIPMENT_DATA.GEO_SID
WHERE
PRODUCT_DETAILS.[NAME] IN ('82281104','82297604','82247351','82247494','82275081','82283786','82242541','82294101','82303374','81690203','81690205','81682913','81682906','81682909','81665060','81729363','81729364','81729365','81689869','81682832','81671867','81677600','81692129','81668240','81538908','81538823','81681079','81681064','81692333','81685653','81687068','80320865','80322948','82284642','82284643','82239084','82243927','82245581','82247350','82283832','82285456','82285457') AND
GEO_LIST.COUNTRY IN ('ISRAEL','THAILAND','INDONESIA','MALAYSIA','Philippines','Singapore','POLAND','ROMANIA','FRANCE','UNITED KINGDOM','TURKEY','GERMANY','CYPRUS','EGYPT','MEXICO','BRAZIL','CHINA','INDIA')  AND
SHIPMENT_DATA.DATE_ID BETWEEN 20190101 AND  20191031  
GROUP BY PRODUCT_DETAILS.[NAME],PRODUCT_DETAILS.[DESCRIPTION],PRODUCT_DETAILS.[ITEM_GTIN], GEO_LIST.COUNTRY
"""

In [5]:
from query_converter.config import load_template, load_css, load_mapping
from query_converter.functions.html_parsing import full_mapping_to_table


css = load_css()
template_str = load_template("converter_template.tmpl")
mapping_df = load_mapping("mapping_2025_09_02.xlsx")
mapping_html = full_mapping_to_table(mapping_df)

In [6]:
from query_converter.functions.transpile import replace_legacy_with_cdl,get_cdl_values, convert_to_databricks, build_mapping_and_comments
from query_converter.functions.helpers import extract_and_qualify

from jinja2 import Template
from query_converter.functions.html_parsing import (
    comment_to_table,
    error_to_table,
    column_mapping_to_table,
    table_mapping_to_table,
    pretty_print_sql,
    highlight_sql_errors,
)
from IPython.core.display import display, HTML

if legacy_query.strip() == "":
    display(HTML("<p>Please enter a query above and run this cell.</p>"))
else:



    result = replace_legacy_with_cdl(legacy_query, mapping_df, catalog="tetette")

    comments_html = comment_to_table(result.comments)
    errors_html = error_to_table(result.errors)
    columns_html = column_mapping_to_table(result.column_mapping)
    errors = list(result.errors.keys()) if result.errors else []

    error_columns = [
        col
        for col, info in (result.errors or {}).items()
        if info.get("error_type") == "column"
    ]

    columns_count = len(result.column_mapping) if result.column_mapping else 0
    comments_count = len(result.comments) if result.comments else 0
    errors_count = len(result.errors) if result.errors else 0
    tables_html = table_mapping_to_table(result.table_mapping)
    tables_count = len(result.table_mapping) if result.table_mapping else 0

    template = Template(str(template_str))



    highlighted_sql = highlight_sql_errors(
        pretty_print_sql(result.query),
        error_columns=[col for col, info in result.errors.items() if info.get("error_type") == "column"],
        error_tables=[tbl for tbl, info in result.errors.items() if info.get("error_type") == "table"]
    )



    html_output = template.render(
        css=css,
        legacy_query=pretty_print_sql(legacy_query),
        converted_query=highlighted_sql,
        comments_html=comments_html,
        errors_html=errors_html,
        columns_html=columns_html,
        tables_html=tables_html,
        columns_count=columns_count,
        tables_count=tables_count,
        mapping_html=mapping_html,
        comments_count=comments_count,
        errors_count=errors_count,
        error_columns=errors,
    )
    # displayHTML(html_output)
    display(HTML(html_output))

C:\Users\damian.petta\AppData\Local\Temp\ipykernel_22584\1712787029.py:13: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


Legacy Column,CDL-STC Column
wmw.dbo.product_d.name,tetette.gold_stc_masterdata.finish_prod_dim.finish_prod_id
wmw.dbo.product_d.description,tetette.gold_stc_masterdata.finish_prod_dim.finish_prod_long_name
wmw.dbo.product_d.item_gtin,tetette.gold_stc_masterdata.finish_prod_dim.item_gtin
wmw.dbo.geo_d.country,tetette.gold_stc_masterdata.geo_dim.iso_country_name
wmw.dbo.product_d.product_sid,tetette.gold_stc_masterdata.finish_prod_dim.prod_key
wmw.dbo.ship_prod_f.product_sid,tetette.gold_ship.ship_mth_fct.finish_prod_id
wmw.dbo.geo_d.geo_sid,tetette.gold_stc_masterdata.geo_dim.geo_key
wmw.dbo.ship_prod_f.geo_sid,tetette.gold_ship.ship_mth_fct.ship_to_management_geo_key
wmw.dbo.ship_prod_f.vol_stat_unit,tetette.gold_ship.ship_mth_fct.stat_unit_qty
wmw.dbo.ship_prod_f.vol_base_unit,tetette.gold_ship.ship_mth_fct.base_unit_qty
